# Генерация из GPT

Будем генерировать твиты по примерам из аккаунта "Усы Пескова": https://twitter.com/Sandy_mustache

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 4.3 MB/s 
     |████████████████████████████████| 596 kB 41.2 MB/s 
     |████████████████████████████████| 3.3 MB 37.2 MB/s 
     |████████████████████████████████| 61 kB 463 kB/s 
     |████████████████████████████████| 895 kB 38.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!wget https://gist.githubusercontent.com/avidale/d3da0ded85a4a16db6eb84d8331638ce/raw/a188084e5ef37b43b01fef0534b55c865b9a569e/tweets.txt

--2021-12-01 18:13:21--  https://gist.githubusercontent.com/avidale/d3da0ded85a4a16db6eb84d8331638ce/raw/a188084e5ef37b43b01fef0534b55c865b9a569e/tweets.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4659 (4.5K) [text/plain]
Saving to: ‘tweets.txt’

tweets.txt          100%[===================>]   4.55K  --.-KB/s    in 0s      

2021-12-01 18:13:21 (52.4 MB/s) - ‘tweets.txt’ saved [4659/4659]



In [ ]:
with open('tweets.txt', 'r') as f:
    tweets = f.read().strip().split('\n\n')
print(len(tweets))
for i in range(3):
    print(tweets[i])

26
Соловьев наконец-то вышел на новый уровень - теперь его стали банить и в офлайне
Дарим мы тебе бутылку игристого вина. Пить тебе еще рано, но встретиться с ней за некоторые преступления ты уже можешь. ПОЗ-ДРАВ-ЛЯ-ЕМ!
Да. Еще очень многие помнят, что такое госплан, как планировалось, талоны на еду, очереди, дефицит, выездные визы. Но спасибо, что напомнил


In [ ]:
import torch
device = torch.device("cuda")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
model_name = 'sberbank-ai/rugpt3large_based_on_gpt2'
#model_name = 'Grossmend/rudialogpt3_medium_based_on_gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

Downloading:   0%|          | 0.00/609 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/2.93G [00:00<?, ?B/s]

In [ ]:
import random

In [ ]:
sep = '\n***\n'
# sep = '\n27479153	Sandy_mustache	2021-02-18 16:44:00	'

prefix = sep.join([''] + random.sample(tweets, k=5) + [''])

tokens = tokenizer(prefix, return_tensors='pt')
tokens = {k: v.to(model.device) for k, v in tokens.items()}
end_token_id = tokenizer.encode('***')[0]
print(prefix)


***
Ладно. Уговорили. Закрывайте все магазины, где продают светильники, как пособников вражеской авиации!
***
Есть подозрение, что те самые 400 кг порошка хранятся в кабинете Володина под его чутким контролем. Иначе эти фантазии не объяснить
***
- Скажи что-нибудь сладкое?
- Сахар
- Не. Еще слаще
- Мёд
- ЕЩЕ СЛАЩЕ!!!
- Бюджет!!!
- Вооооот
***
Знаете, это тот случай, когда Россия может на 400% доверять своему партнеру
***
Внимание. В России официально появился парк славы Илона Маска
***



In [ ]:
size = tokens['input_ids'].shape[1]
output = model.generate(
    **tokens, 
    #end_token=end_token_id,
    do_sample=False, 
    max_length=size+128, 
    repetition_penalty=4.2, 
    temperature=0.7,
    num_beams=3,
)
decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


У нас теперь есть новый министр обороны - Сергей Шойгу
***
А у вас какие планы на выходные?<s>
Оригинал взят у в postСмотрю новости и думаю: "Как же так получилось, что я живу в такой стране, где люди живут хуже чем в других странах?" 
И тут меня осенило... 

Это потому, что мы с вами живем в очень богатой стране. У нас богатая история, культура, наука, промышленность, сельское хозяйство, образование, здравоохранение, армия, флот, авиастроение, атомная энергетика, космическая техника, ядерное оружие, космос, авиация,


# Диалоги

Логика та же самая, что и с твитами. Только выбрали разделитель в стиле bash.org

In [ ]:
def respond_to_dialog(texts):
    prefix = '\nx:'
    for i, t in enumerate(texts):
        prefix += t
        prefix += '\nx:' if i % 2 == 1 else '\ny:'
    tokens = tokenizer(prefix, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}
    end_token_id = tokenizer.encode('\n')[0]
    size = tokens['input_ids'].shape[1]
    output = model.generate(
        **tokens, 
        eos_token_id=end_token_id,
        do_sample=True, 
        max_length=size+128, 
        repetition_penalty=3.2, 
        temperature=1,
        num_beams=3,
        length_penalty=0.01,
        pad_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.decode(output[0])
    result = decoded[len(prefix):]
    return result.strip()

In [ ]:
seed = input('Начните диалог с ботом любой фразой\n')
history = [seed]
while True:
    result = respond_to_dialog(history[-10:])
    next_sentence = input(result + '\n')
    history.append(result)
    history.append(next_sentence)

Начните диалог с ботом любой фразой
Привет


/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:2142: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


Как дела?
а у тебя?
У меня все хорошо!
что делаешь?


KeyboardInterrupt: ignored

In [ ]:
for i in range(10):
    print(respond_to_dialog(['Давай поговорим о домашних животных', 'Каких питомцев вы любите?\n--------', 
                             'Давай поговорим о машинах', 'Какого цвета твой автомобиль?\n--------',
                             'Давай поговорим о физике']))

Как ты относишься к музыке?
Какие предметы ты любишь?
А что это такое?
Как ты относишься к теории струн?
Что такое электричество?
Каким образом ты меняешь цвет автомобиля?
Какое у тебя хобби?
Что такое электричество?
Какое животное тебе больше всего нравится?
Что такое электричество?
